# RSI Indicator analysis


In [2]:
%load_ext memory_profiler

In [3]:
import pandas as pd 
import numpy as np
import time
%memit np.zeros(100000)
%timeit np.zeros(100000)

peak memory: 99.36 MiB, increment: 0.96 MiB
13.3 µs ± 202 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [4]:
n = 14
date = range(1,39)
close = [54.80,56.80,57.85,59.85,60.57,61.10,62.17,60.60,62.35,62.15,62.35,61.45,62.80,61.37,62.50,62.57,60.80,59.37,60.35,62.35,62.17,62.55]
close += [64.55,64.37,65.30,64.42,62.90,61.60,62.05,60.05,59.70,60.90,60.25,58.27,58.70,57.72,58.10,58.20]
data = {'date': date, 'close': close}

df = pd.DataFrame(data)

In [11]:
df = pd.read_csv("data/input/USDT_BTC_86400_1425168000_72.csv")
print(f"Dataframe shape:{df.shape}")
df = df.drop(columns=['ts','period','currency_pair','volume','quoteVolume','quoteVolume','weightedAverage'])
df.head()

Dataframe shape:(2192, 11)


,date,high,low,open,close
0,1425168000,251.00,240.000121,240.000121,251.00000
1,1425254400,269.00,251.000000,252.000000,269.00000
2,1425340800,274.50,255.000000,268.000000,267.22000
3,1425427200,280.00,258.875000,258.875000,280.00000
4,1425513600,280.89,257.000000,257.000000,257.00012


In [5]:
def get_rsi_v0(series, period): 
    
    delta = series.diff().dropna()
    
    u = delta.where(delta > 0).fillna(0)
    d = delta.where(delta < 0).fillna(0).abs()
   
    u[u.index[period-1]] = np.mean( u[:period] ) 
    u = u.drop(u.index[:(period-1)])
 
    d[d.index[period-1]] = np.mean( d[:period] )
    d = d.drop(d.index[:(period-1)])
    
    u_ema =  u.ewm(com=period-1, adjust=False).mean() # if com=13 alfa=1/14 if adjust=false (1-alfa)= 13/14
    d_ema = d.ewm(com=period-1, adjust=False).mean()
    rsi = u_ema / (u_ema + d_ema)
    return rsi

In [9]:
def get_rsi_v1(df,n):
    
    delta = df.close - df.close.shift(1) 
    delta = delta.dropna()
    
    closeup = delta.mask(delta < 0, 0)
    closedown = delta.mask(delta > 0,0).abs()

    closeup[closeup.index[n-1]]= closeup[:n].mean()
    closeup = closeup.drop(closeup.index[:n-1])
    avg_closeup = closeup.ewm(com=n-1,adjust=False).mean()
  
    closedown[closedown.index[n-1]]= closedown[:n].mean()
    closedown = closedown.drop(closedown.index[:n-1])
    avg_closedown = closedown.ewm(com=n-1,adjust=False).mean()


    rsi = avg_closeup / (avg_closedown + avg_closeup)
    
    return rsi

In [12]:
%timeit df['rsi_v0'] = get_rsi_v0(df.close,n)
%timeit df['rsi_v1'] = get_rsi_v1(df,n)
df

2.28 ms ± 13 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.35 ms ± 3.73 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


,date,high,low,open,close,rsi_v0,rsi_v1
0,1425168000,251.000000,240.000121,240.000121,251.000000,NaN,NaN
1,1425254400,269.000000,251.000000,252.000000,269.000000,NaN,NaN
2,1425340800,274.500000,255.000000,268.000000,267.220000,NaN,NaN
3,1425427200,280.000000,258.875000,258.875000,280.000000,NaN,NaN
4,1425513600,280.890000,257.000000,257.000000,257.000120,NaN,NaN
...,...,...,...,...,...,...,...
2187,1614124800,51347.547800,47012.372297,48893.566626,49694.850275,0.570222,0.570222
2188,1614211200,52019.280000,46719.505800,49678.345446,47071.988756,0.513661,0.513661
2189,1614297600,48448.570888,44122.216031,47046.151566,46310.000000,0.498200,0.498200
2190,1614384000,48388.981486,45000.000000,46320.000000,46121.948326,0.494246,0.494246
